<a href="https://colab.research.google.com/github/t1seo/Python_Notebook/blob/master/effective_python/%EC%BB%B4%ED%94%84%EB%A6%AC%ED%97%A8%EC%85%98%EA%B3%BC%20%EC%A0%9C%EB%84%88%EB%A0%88%EC%9D%B4%ED%84%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CHAPTER 4 컴프리헨션과 제너레이터

- 파이썬에서는 **컴프리헨션(comprehension)**이라는 특별한 구문을 사용해, 리스트, 딕셔너리, 집합 등의 타입을 간결하게 이터레이션하면서 원소로부터 파생되는 데이터 구조르 생성할 수 있다.

- 컴프리헨션 코딩 스타일은 **제너레이터(generator)**를 사용하는 함수로 확장할 수 있다. 제너레이터는 함수가 점진적으로 반환하는 값으로 이뤄지는 스트림을 만들어준다. 이터레이터를 사용할 수 있는 곳(foraㅜㄴ, 별표 식)이라면 어디에서나 제너레이터 함수를 호출한 결과를 사요할 수 있다.


제너레이터를 사용하면 성능을 향상시키고, 메모리 사용을 줄이고, 가독성을 높일 수 있다.

## BETTER WAY 27 map과 filter 대신 컴프리헨션을 사용하라

## BETTER WAY 28 컴프리헨션 내부에 제어 하위 식을 세 개 이상 사용하지 말라

## BETTER WAY 29 대입식을 사용해 컴프리헨션 안에서 반복 작업을 피하라

## BETTER WAY 30 리스트를 반환하기보다는 제너레이터를 사용하라

In [ ]:
# 제너레이터 사용 안한 코드
def index_words(text):
    result = []
    if text:
        result.append(0)
    for index, letter in enumerate(text):
        if letter == ' ':
            result.append(index + 1)
    return result

address = '컴퓨터(영어: Computer, 문화어: 콤퓨터, 순화어: 전산기)는 진공관'
result = index_words(address)
print(result)

In [ ]:
# 제너레이터 사용
def index_words_iter(text):
    if text:
        yield 0
    for index, letter in enumerate(text):
        if letter == ' ':
            yield index + 1

- 이 함수가 호출되면 제너레이터 함수가 실행되지 않고 즉시 이터레이터를 반환한다.
- 이터레이터가 `next` 내장 함수를 호출할 때마다 이터레이너튼 제너레이터 함수를 다음 `yield` 식까지 진행시킨다.
- 제너레이터가 `yield`에 전달하는 값은 이터레이터에 의해 호출하는 쪽에 반환된다.

In [ ]:
it = index_words_iter(address)
print(next(it))
print(next(it))

In [ ]:
result = list(index_words_iter(address)) # 이터레이터를 리스트로
print(result)

In [ ]:
# 파일에서 한 번에 한 줄씩 읽어 한 단어씩 출력하는 제너레이터
def index_file(handle):
    offset = 0
    for line in handle:
        if line:
            yield offset
        for letter in line:
            offset += 1
            if letter == ' ':
                yield offset
            

In [ ]:
from google.colab import files
uploaded = files.upload() # 파일 업로드 기능 실행

In [ ]:
import itertools

with open('lorem_ipsum.txt', 'r', encoding='utf-8') as f:
    # lorem_ipsum = f.read()
    # print(lorem_ipsum)
    it = index_file(f)
    results = itertools.islice(it, 0, 10)
    print(list(results))

## BETTER WAY 31 인자에 대해 이터레이션할 때는 방어적이 돼라

In [ ]:
# 정규화 함수
def normalize(numbers):
    total = sum(numbers) # 이터레이터 사용 1
    result = []
    for value in numbers: # 이터레이터 사용 2
        percent = 100 * value / total # 각각 x 100 / 총합 => normalization
        result.append(percent)
    return result

visits = [15, 35, 80]
percentages = normalize(visits)
print(percentages)
assert sum(percentages) == 100.0

In [ ]:
from google.colab import files
uploaded = files.upload() # 파일 업로드 기능 실행

In [ ]:
def read_visits(data_path):
    with open(data_path) as f:
        for line in f:
            yield int(line) # 이터레이터 한 번 사용

it = read_visits('numbers.txt')
percentages = normalize(it) # 이터레이터 두 번 사용
print(percentages) # 아무런 결과가 출력이 안된다

- 위 코드는 실행이 안된다. normalize 함수에서 `sum`과 `for`에서 이터레이터를 두 번 사용하기 때문이다.

- 이터레이터는 결과를 단 한번만 만들어낸다. 이미 `StopIteration` 예외가 발생한 이터레이터나 제너레이터를 다시 이터레이션하면 아무런 결과도 얻을 수 없다.




해결 방법:
- 이터레이터를 명시적으로 소진시키고 이터레이터의 전체 내용을 리스트에 넣는다
    - 메모리를 엄청 많이 사용할 수 있다.
- 호출될 때마다 새로운 이터레이터를 반환하는 함수를 받는다.
    - 보기에 좋지 않다.
- **이터레이터 프로토콜(iterator protocol)**
    - 이터레이터 프로토콜은 파이썬의 `for`루프나 그와 연관된 식들이 컨테이너 타입의 내용을 방문할 때 사용하는 절차이다.

In [ ]:
class ReadVisits:
    def __init__(self, data_path):
        self.data_path = data_path
    
    def __iter__(self):
        with open(self.data_path) as f:
            for line in f:
                yield int(line)

In [ ]:
visits = ReadVisits('numbers.txt')
percentages = normalize(visits)
print(percentages)
assert sum(percentages) == 100.0

- 이 코드가 잘 작동하는 이유는 `normalize` 함수 안의 `sum` 메서드가 `ReadVisits.__iter__`를 호출해서 새로운 이터레이터 객체를 할당하기 때문이다.
- 각 숫자를 정규화하기 위한 `for` 루프도 `__iter__`를 호출해서 두 번째 이터레이터 객체를 만든다.

- 두 이터레이터는 서로 독립적으로 진행되고 소진된다. 이로 인해 두 이터레이터는 모든 입력 데이터 값을 볼 수 있는 별도의 이터레이터를 만들어낸다.

### 인자가 이터레이터인지 검사

In [ ]:
def normalize_defensive(numbers):
    if iter(numbers) is numbers: # is를 사용해 iterator인지 검사
        raise TypeError('컨테이너를 제공해야 합니다')
    total = sum(numbers)
    result = []
    for value in numbers:
        percent = 100 * value / total
        result.append(percent)
    return result

`collections.abc` 내장 모듈은 `isinstance`를 사용해 잠재적인 문제를 검사할 수 있는 `Iterator` 클래스를 제공한다.

In [ ]:
from collections.abc import Iterator

def normalize_defensive(numbers):
    if isinstance(numbers, Iterator): # 반복 가능한 이터레이터인지 검사하는 다른 방법
        raise TypeError('컨테이너를 제공해야 합니다')
    total = sum(numbers)
    result = []
    for value in numbers:
        percent = 100 * value / total
        result.append(percent)
    return result

In [ ]:
visits = [15, 35, 80]
percentages = normalize_defensive(visits)
assert sum(percentages) == 100.0

visits = ReadVisits('number.txt')
percentages = normalize_defensive(visits)
assert sum(percentages) == 100.0

#
visits = [15, 35, 80]
it = iter(visits)
# 오류가 나는 부분. 오류를 보고 싶으면 커멘트를 해제할것
#normalize_defensive(it)

- 이 함수는 입력이 컨테이너가 아닌 이터레이터면 예외를 발생시킨다.
- 비동기 이터레이터에 대해서도 같은 접근 방식을 사용할 수 있다.